In [3]:
!pip install pymystem3
!pip install nltk

# Устанавливаем нужные библиотеки
!pip install pymystem3
!pip install nltk

# Импортируем библиотеки
from pymystem3 import Mystem
import re

# Создаем анализатор для приведения слов к начальной форме
mystem = Mystem()

def get_text_features(text):
    """
    Извлекает простые признаки из текста
    Возвращает словарь с 4 признаками
    """
    # 1. Очищаем текст
    text = text.lower()
    text = text.replace('ё', 'е')
    text = re.sub(r'[^а-яё\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 2. Делим на слова
    words = text.split()

    if not words:  # Если текст пустой
        return {
            'avg_word_len': 0,
            'unique_ratio': 0,
            'long_words_ratio': 0,
            'stopwords_ratio': 0
        }

    # 3. Считаем признаки

    # Признак 1: Средняя длина слова
    avg_word_len = sum(len(word) for word in words) / len(words)

    # Признак 2: Процент уникальных слов
    unique_ratio = len(set(words)) / len(words)

    # Признак 3: Процент длинных слов (больше 6 букв)
    long_words = [word for word in words if len(word) > 6]
    long_words_ratio = len(long_words) / len(words)

    # Признак 4: Процент стоп-слов (самых частых русских слов)
    common_words = {'и', 'в', 'не', 'на', 'я', 'быть', 'с', 'что', 'а', 'по', 'это', 'он', 'к', 'но', 'они', 'мы', 'как', 'у', 'вы', 'от', 'за', 'же', 'из', 'ли', 'то', 'все', 'о', 'при', 'так', 'же'}
    stopwords_count = sum(1 for word in words if word in common_words)
    stopwords_ratio = stopwords_count / len(words)

    # Возвращаем признаки
    return {
        'avg_word_len': avg_word_len,
        'unique_ratio': unique_ratio,
        'long_words_ratio': long_words_ratio,
        'stopwords_ratio': stopwords_ratio
    }


def train_genre_classifier():
    """
    Создает 'мозг' алгоритма - запоминает признаки разных жанров
    Возвращает 'базу знаний' о жанрах
    """
    # Это наши "учебные материалы" - примеры текстов каждого жанра
    training_texts = {
        'художественный': [
            "Лейтенант шел по желтому строительному песку, нагретому дневным палящим солнцем. Он был мокрым от кончиков пальцев до кончиков волос, все его тело было усеяно царапинами от острой колючей проволоки и ныло от сводящей с ума боли, но он был жив и направлялся к командному штабу, который виднелся на горизонте метрах в пятистах.", "Яша был всего лишь мелким пакостником, который, тем не менее, имел очень большой потенциал. Еще в розовом детстве он виртуозно тырил яблоки у тети Нюры, а не прошло и каких-то двадцати лет, как он с тем же лихим запалом переключился на банки в двадцати трех странах мира, причем умудрялся так мастерски их обчищать, что ни полиция, ни Интерпол никак не могли взять его с поличным.", "Варенька, такая милая, добродушная и отзывчивая девушка, глаза которой всегда лучились добротой и теплом, с невозмутимым видом сущего демона шла к бару «Гадкий Гарри» с автоматом Томпсона наперевес, готовая укатать в асфальт этих мерзких, грязных, вонючих и скользких типов, посмевших пялиться на ее прелести и пускать похотливые слюни."
        ],
        'научный': [
            "Исходя из результатов эксперимента, можно сделать заключение, что объект имеет мягкую однородную структуру, свободно пропускает свет и может изменять ряд своих параметров при воздействии на него разности потенциалов в диапазоне от 5 до 33 000 В. Исследования также показали, что объект необратимо изменяет свою молекулярную структуру под воздействием температуры свыше 300 К. При механическом воздействии на объект с силой до 1000 Н видимых изменений в структуре не наблюдается.", "Среди феноменологических моделей наибольшее распространение получили модели основанные на лучевых представлениях полей рассеяния и модели базирующиеся на принципе Гюйгенса-Френеля, согласно которому каждая точка волнового фронта рассеянного поля рассматривается как источник вторичных волн. К этому типу моделей относится модель локальных источников рассеяния. Несмотря на то, что все эти модели имеют одни и те же методологические основы, они имеют особенности, которые проявляются при дополнительных упрощениях и допущениях математического и физического характера. Аналоговой моделью принято называть стохастическую модель «блестящих» точек, получивую широкое распространение при анализе рассеянных полей от объектов, которые представлены в виде совокупности локальных отражателей."
        ],
        'публицистический': [
            "Невероятное открытие! Житель глухой деревни Эксперименталово изобрел новый препарат, заставляющий куриц нести золотые яйца! Тайна, над которой не одно столетие бились величайшие алхимики мира, наконец, раскрыта нашим соотечественником! Пока от изобретателя никаких комментариев не поступало, он, в данный момент, находится в сильном запое, однако можно однозначно сказать, что открытия таких патриотов, однозначно, стабилизируют экономику нашей страны и укрепят ее позиции на мировой арене как лидера в области добычи золота и производстве золотых изделий на десятки лет вперед.", "Можно ли давать урок, если нет классной комнаты? Сдавать экзамен без зачетки в кармане? Входит ли вдыхание меловой пыли полной грудью необходимой составной частью в процесс получения высшего образования? Вот Виктор В., например, так не думает. Ему 41 год, он не последний человек в одной из страховых компаний и в то же время – студент университета третьего возраста. Во время перерыва на обед или вечером, когда жена Виктора уже спит, он садится за клавиатуру компьютера, входит в Интернет и начинает грызть гранит науки в том месте, которое ему обозначил профессор, стоящий на кафедре в семи сотнях километров от его дома."
        ],
        'разговорный': [
            "Вы когда нибудь задумывалить о том, что было бы, если бы Земля поменялась местами с Юпитером? Я серьезно! Возникли бы Новые Васюки на его кольцах? Конечно нет! Они же из газа! Неужели вы хоть на минуту купились на такую откровенную чушь? В жизни не поверю! А если бы луна упала в Тихий Океан, на сколько бы поднялся его уровень? Вы, наверное, думаете, что я – редкий зануда, но, если я не задам эти вопросы, то кто?", "Стыдно, женка. Ты на меня сердишься, не разбирая, кто виноват, я или почта, и оставляешь меня две недели без известия о себе и о детях. Я так был смущен, что не знал, что и подумать. Письмо твое успокоило меня, но не утешило. Описание вашего путешествия в Калугу, как ни смешно, для меня вовсе не забавно. Что за охота таскаться в скверный уездный городишко, чтоб видеть скверных актеров, скверно играющих старую, скверную оперу? <...> Просил я тебя по Калугам не разъезжать, да, видно, уж у тебя такая натура."
        ]
    }

    # Собираем "базу знаний" о каждом жанре
    genre_knowledge = {}

    for genre, texts in training_texts.items():
        # Собираем все признаки для всех текстов этого жанра
        all_features = []
        for text in texts:
            features = get_text_features(text)
            all_features.append(features)

        # Находим СРЕДНИЕ значения признаков для этого жанра
        avg_features = {
            'avg_word_len': sum(f['avg_word_len'] for f in all_features) / len(all_features),
            'unique_ratio': sum(f['unique_ratio'] for f in all_features) / len(all_features),
            'long_words_ratio': sum(f['long_words_ratio'] for f in all_features) / len(all_features),
            'stopwords_ratio': sum(f['stopwords_ratio'] for f in all_features) / len(all_features)
        }

        genre_knowledge[genre] = avg_features

    return genre_knowledge


def predict_genre(text, genre_knowledge):
    """
    Определяет жанр нового текста
    Сравнивает его признаки с 'базой знаний' о жанрах
    """
    # 1. Извлекаем признаки из нового текста
    text_features = get_text_features(text)

    # 2. Сравниваем с каждым жанром из базы знаний
    distances = {}

    for genre, genre_features in genre_knowledge.items():
        # Считаем "расстояние" между текстом и жанром
        # Чем меньше расстояние - тем больше похож

        distance = 0
        # Сравниваем каждый признак
        for key in text_features.keys():
            # Разница по каждому признаку
            diff = abs(text_features[key] - genre_features[key])
            distance += diff

        distances[genre] = distance

    # 3. Находим самый похожий жанр (с наименьшим расстоянием)
    predicted_genre = min(distances, key=distances.get)

    # 4. Возвращаем результат
    return predicted_genre, distances


def main():
    """
    Главная функция - запускает весь алгоритм
    """
    print("=" * 60)
    print("АЛГОРИТМ ОПРЕДЕЛЕНИЯ ЖАНРА ТЕКСТА")
    print("=" * 60)

    # 1. Обучаем алгоритм (создаем базу знаний)
    print("\n1. Обучаю алгоритм на примерах...")
    genre_knowledge = train_genre_classifier()

    print("База знаний создана! Знаю 4 жанра:")
    for genre in genre_knowledge:
        print(f"  - {genre}")

    # 2. Получаем текст от пользователя
    print("\n2. Введите текст для анализа:")
    print("(Можно вставить любой русский текст)")
    print("-" * 40)

    # Пример текста для тестирования (пользователь заменит на свой)
    user_text = """Пьер так и не успел выбрать себе карьеры в Петербурге и действительно был выслан в Москву за буйство. История, которую рассказывали у графа Ростова, была справедлива. Пьер участвовал в связыванье квартального с медведем. Он приехал несколько дней тому назад и остановился, как всегда, в доме своего отца. Хотя он и предполагал, что история его уже известна в Москве и что дамы, окружающие его отца, всегда недоброжелательные к нему, воспользуются этим случаем, чтобы раздражить графа, он все-таки в день приезда пошел на половину отца. Войдя в гостиную, обычное местопребывание княжон, он поздоровался с дамами, сидевшими за пяльцами и за книгой, которую вслух читала одна из них. Их было три."""

    print(f"Текст для анализа:\n{user_text}")

    # 3. Определяем жанр
    print("\n3. Анализирую текст...")
    predicted_genre, distances = predict_genre(user_text, genre_knowledge)

    # 4. Показываем результат
    print("\n4. РЕЗУЛЬТАТ АНАЛИЗА:")
    print("=" * 40)
    print(f"Определенный жанр: {predicted_genre.upper()}")
    print("\nНасколько текст похож на каждый жанр:")
    print("(чем меньше число - тем больше похож)")
    print("-" * 40)

    for genre, distance in sorted(distances.items(), key=lambda x: x[1]):
        similarity = max(0, 100 - distance * 20)  # Преобразуем в проценты
        print(f"{genre:20} - {distance:.3f} ({similarity:.0f}% похоже)")

    print("=" * 60)


# Запускаем программу
if __name__ == "__main__":
    main()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


АЛГОРИТМ ОПРЕДЕЛЕНИЯ ЖАНРА ТЕКСТА

1. Обучаю алгоритм на примерах...
База знаний создана! Знаю 4 жанра:
  - художественный
  - научный
  - публицистический
  - разговорный

2. Введите текст для анализа:
(Можно вставить любой русский текст)
----------------------------------------
Текст для анализа:
Пьер так и не успел выбрать себе карьеры в Петербурге и действительно был выслан в Москву за буйство. История, которую рассказывали у графа Ростова, была справедлива. Пьер участвовал в связыванье квартального с медведем. Он приехал несколько дней тому назад и остановился, как всегда, в доме своего отца. Хотя он и предполагал, что история его уже известна в Москве и что дамы, окружающие его отца, всегда недоброжелательные к нему, воспользуются этим случаем, чтобы раздражить графа, он все-таки в день приезда пошел на половину отца. Войдя в гостиную, обычное местопребывание княжон, он поздоровался с дамами, сидевшими за пяльцами и за книгой, которую вслух читала одна из них. Их было три.

3. Ан